In [189]:
import pandas as pd
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import ConvLSTM2D
#from sklearn

In [190]:
#!pip install keras
#!pip install tensorflow

In [191]:
xls = pd.ExcelFile('2 years data.xlsx',engine='openpyxl')
df1 = pd.read_excel(xls, 'Planilha1')
df2 = pd.read_excel(xls, 'Price increase dates')
df3 = pd.read_excel(xls, 'Mandatory vacation dates',names=['start','end'])

In [192]:
df1 = df1.iloc[:245815]
df2.dropna(axis=1,inplace=True)
df3.dropna(inplace=True)
df3.drop([0],inplace=True)

In [193]:
df1.drop(['Order nº','Custumer Name','Item Name','Subgroup_description'],axis=1,inplace=True)
df2.drop(['Subgroup of products (description)'],axis=1,inplace=True)

In [194]:
df1

,Order date,Custumer Id,State,City,Country,Item Id,Quantity,Subgruop_ID
0,2019-10-01,195.0,SP,CAMPINAS,BRASIL,30510018.0,3.0,51.0
1,2019-10-01,195.0,SP,CAMPINAS,BRASIL,30510019.0,2.0,51.0
2,2019-10-01,195.0,SP,CAMPINAS,BRASIL,30600083.0,10.0,60.0
3,2019-10-01,195.0,SP,CAMPINAS,BRASIL,30610099.0,1.0,61.0
4,2019-10-01,195.0,SP,CAMPINAS,BRASIL,30610071.0,1.0,61.0
...,...,...,...,...,...,...,...,...
245810,2021-09-13,5000.0,EX,MONTEVIDEO,URUGUAI,30510074.0,40.0,51.0
245811,2021-09-13,5000.0,EX,MONTEVIDEO,URUGUAI,30510075.0,80.0,51.0
245812,2021-09-13,5000.0,EX,MONTEVIDEO,URUGUAI,30510189.0,20.0,51.0
245813,2021-09-13,5000.0,EX,MONTEVIDEO,URUGUAI,30530005.0,20.0,53.0


In [195]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in ['Country','State','City']:
    df1[col] = le.fit_transform(df1[[col]])

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [196]:
df = df1.groupby(['Order date','Custumer Id','Item Id','Country','State','City','Subgruop_ID']).sum().reset_index()

In [197]:
df

,Order date,Custumer Id,Item Id,Country,State,City,Subgruop_ID,Quantity
0,2019-10-01,62.0,30510001.0,1,12,42,51.0,50.0
1,2019-10-01,62.0,30510004.0,1,12,42,51.0,6.0
2,2019-10-01,62.0,30510007.0,1,12,42,51.0,3.0
3,2019-10-01,62.0,30510026.0,1,12,42,51.0,10.0
4,2019-10-01,62.0,30520004.0,1,12,42,52.0,50.0
...,...,...,...,...,...,...,...,...
244855,2021-09-13,5000.0,30710008.0,8,8,126,71.0,10.0
244856,2021-09-13,5000.0,30710011.0,8,8,126,71.0,30.0
244857,2021-09-13,5000.0,30710014.0,8,8,126,71.0,5.0
244858,2021-09-13,8112.0,30060030.0,1,23,36,6.0,4000.0


In [198]:
new_data = pd.DataFrame(pd.date_range(df['Order date'].min(),df['Order date'].max(),freq='d'),columns=['Date'])
new_data = pd.merge(new_data,df.loc[df['Custumer Id']==270][['Order date']].drop_duplicates(),
                    left_on='Date',right_on='Order date',how='outer')
new_data['Buy']= new_data['Order date'].isna()==0
new_data['Buy'].replace({True:1,False:0},inplace=True)
new_data.drop(['Order date'],axis=1,inplace=True)

new_data = pd.merge(new_data,df2[['Date of price increase']].drop_duplicates(),
                    left_on='Date',right_on='Date of price increase',how='outer')
new_data['Price increase']= new_data['Date of price increase'].isna()==0
new_data['Price increase'].replace({True:1,False:0},inplace=True)
new_data.drop(['Date of price increase'],axis=1,inplace=True)

new_data['Vacation'] = 0
for i in range(1,1+len(df3)):
    new_data = pd.merge(new_data,pd.DataFrame(pd.date_range(df3['start'][i],df3['end'][i],freq='d'),columns=['vDate']),left_on='Date',right_on='vDate',how='outer')
    new_data['vDate']= new_data[['vDate']].isna()==0
    new_data['vDate'].replace({True:1,False:0},inplace=True)
    new_data['Vacation'] = new_data['Vacation'] + new_data['vDate']
    new_data.drop(['vDate'],axis=1,inplace=True)

new_data.drop(['Date'],axis=1,inplace=True)
new_data

,Buy,Price increase,Vacation
0,1,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
709,0,0,0
710,0,0,0
711,0,0,0
712,0,0,0


In [229]:
n_steps_in = 300
n_features = 2
n_steps_out=60

In [219]:
def split_sequences(sequences, n_steps_in=300, n_steps_out=60):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out-1
		# check if we are beyond the dataset
		if out_end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [220]:
x,y = split_sequences(new_data.to_numpy(),)

In [269]:
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps_in, 2)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(x, y, epochs=5, validation_split=.8, verbose=4)

Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5


In [278]:
model.predict(x[0].reshape(-1,300,2))

array([[-2.3586513e-04, -7.4586726e-04,  3.9605459e-04, -2.6173555e-04,
        -4.5551299e-04, -1.7432799e-04, -3.8193481e-04, -3.0755741e-04,
         4.3730508e-04,  4.7204783e-05,  9.9859631e-04,  4.7499861e-04,
         3.2049709e-04,  3.4994446e-05,  1.5067172e-04,  1.3471340e-03,
         2.0629843e-05, -3.8114924e-04, -2.0828785e-04,  1.0622125e-03,
        -1.8165156e-04, -4.1521247e-04, -8.4402441e-04, -4.3302443e-04,
         2.9551610e-04, -8.3087775e-04, -1.0670398e-03, -1.6277249e-04,
        -5.8781414e-04, -3.2979905e-04, -2.1693361e-04,  3.2625906e-04,
         3.6530895e-04, -5.6326500e-04, -1.8691650e-04, -3.0715798e-04,
         7.0634478e-04,  6.3676934e-04,  4.4825138e-04, -6.1637897e-04,
         3.8566277e-04, -5.0329731e-04,  2.0842988e-04,  2.2556481e-04,
        -1.1741910e-03,  4.1764753e-04, -7.7589101e-04, -1.7985213e-04,
         4.0155719e-05, -3.1570124e-04, -2.4576078e-04, -6.0511293e-04,
         4.3337501e-04, -3.3439061e-04, -8.6167012e-04,  5.14242